<a href="https://colab.research.google.com/github/Mathijsgeelen/Sentence-extraction-annotation/blob/main/Train_BERT_model_SBD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Required installations

In [1]:
pip install transformers

     |████████████████████████████████| 3.4 MB 4.1 MB/s 
     |████████████████████████████████| 895 kB 56.2 MB/s 
     |████████████████████████████████| 3.3 MB 44.7 MB/s 
     |████████████████████████████████| 61 kB 388 kB/s 
     |████████████████████████████████| 596 kB 55.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
pip install sentencepiece

     |████████████████████████████████| 1.2 MB 4.1 MB/s 


In [3]:
pip install torch

In [4]:
pip install seqeval

     |████████████████████████████████| 43 kB 1.3 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=4fcb2f05a2f80bd41bdd6f94912ceeedea46fecdf96ae30f66db9c22c716be7f
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [5]:
pip install wandb

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
     |████████████████████████████████| 180 kB 52.1 MB/s 
     |████████████████████████████████| 140 kB 55.2 MB/s 
     |████████████████████████████████| 97 kB 7.2 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=732eec93e996126ee02e52d492588384c54b55cc1a90fbf5b669bb1780719667
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=4eb6f7677d31728009f78091f85bb832c4cb175539feb186d0df6c466e2277dc
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


# Required imports

In [6]:
import torch
from torch.optim import Adam, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
from transformers import AutoModelForTokenClassification, AdamW
import random
import os
import tensorflow as tf
import numpy as np
from seqeval.metrics import classification_report, f1_score, accuracy_score
from transformers import DebertaTokenizer, DebertaForTokenClassification
import wandb


In [7]:
seed = 1
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)
TF_DETERMINISTIC_OPS=seed
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found. Please change to a GPU environment')

Found GPU at: /device:GPU:0


# Instantiate desired Transformer model
Change the below model and tokenizer if you want to train any other model than DeBERTa

In [ ]:
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")
model = DebertaForTokenClassification.from_pretrained("microsoft/deberta-base", num_labels=4,output_attentions=True)

# Mount google drive containing the 4 file of Savelka et al. (2017).
Savelka, Jaromir, Vern R. Walker, Matthias Grabmair and Kevin D. Ashley. "Sentence Boundary Detection in Adjudicatory Decisions in the Uniter States." TAL 58.2 (2017).

In [9]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


Save all the files in your google drive in a folder named SBD, or change the file paths in the below imports

In [10]:
import json
with open("/content/drive/My Drive/SBD/bva.json.txt") as file:
  data_bva = json.load(file)
with open("/content/drive/My Drive/SBD/cyber_crime.json.txt") as file:
  data_cyber = json.load(file)
with open("/content/drive/My Drive/SBD/intellectual_property.json.txt") as file:
  data_ip = json.load(file)
with open("/content/drive/My Drive/SBD/scotus.json.txt") as file:
  data_scotus = json.load(file)

# Information on dataset

In [ ]:
def extract_info():
  ### Extracts information on the lenght of the sentences in the dataset
  names = ["BVA","Cyber","IP","Scotus"]
  av = []
  for name, data in enumerate([data_bva, data_cyber, data_ip, data_scotus]):
    low = 10000
    high = 0
    average = []
    for counter, doc in enumerate(data):
      document = data[doc]
      for i in range(len(document["annotations"])):
        start = document["annotations"][i]["start"]
        end = document["annotations"][i]["end"]
        sent = document["text"][start:end+1]
        len_sent = len(sent.split(" "))
        if len_sent < low:
          low = len_sent
        if len_sent > high:
          high = len_sent
        average.append(len_sent)
        av.append(len_sent)
    print("Dataset: {}; Smallest sentence: {}, Longest sentence: {}, Average sentence: {:.1f}".format(names[name],low,high, np.mean(average)))
    print()
  print("Average sentence lenght of entire dataset: {:.1f}".format(np.mean(av)))

extract_info()

In [ ]:
def ends_with_punctuation():
  ### extracts information on the number of sentences and characters in the dataset
  total_count = 0
  total_sentences = 0
  names = ["BVA","Cyber","IP","Scotus"]
  for name, data in enumerate([data_bva, data_cyber, data_ip, data_scotus]):
    count = 0
    sentences = 0
    for counter, doc in enumerate(data):
      document = data[doc]
      sentences += len(document["annotations"])
      total_sentences += len(document["annotations"])
      for i in range(len(document["annotations"])):
        end = document["annotations"][i]["end"]
        token = document["text"][end-1]
        if token == "." or token == "?" or token == "!":
          count+=1
          total_count += 1
        
    print("Percentage of sentences that end with punctuation mark in {}: {:.3f}".format(names[name],count/sentences))
  print("Percentage of sentences that end with punctuation mark in total dataset: {:.3f}".format(total_count/total_sentences))


ends_with_punctuation()

# Functions to create the dataset

In [ ]:
def create_sentences():
  #create dataset consisting of the individual documents with accompanying sentences split using the annotations.
  sentences = []
  tags = []
  for data in [data_bva, data_cyber, data_ip, data_scotus]:
    for counter, doc in enumerate(data):
      document = data[doc]
      for i in range(len(document["annotations"])):
        start = document["annotations"][i]["start"]
        end = document["annotations"][i]["end"]

        prev_start = document["annotations"][i-1]["start"]
        prev_end = document["annotations"][i-1]["end"]
        sent = document["text"][start:end+1]
        prev_sent = document["text"][prev_start:prev_end+1]
        if prev_sent[-1] == " ":
          sent = " " + sent
        if sent[-1] == " ":
          sent = sent[:-1]
        sentences.append(sent)

  return sentences

print("Complete dataset contains {} sentences".format(len(create_sentences())))

print("This function should return 25899")

In [14]:
def concatenate_sentences(extracted_sentences):
  ### this function loops over the individual sentences and concatenates them to create batches on lenght 512 tokens
  MAX_LEN = 510
  mas_sequence_list = []
  mas_label_list = []
  ct_sequence_length = 0
  ct_seq_list = []
  ct_token_list = []
  original_sent = []
  tokenized_sent = []
  for i, ind_sent in enumerate(extracted_sentences):
    # add each sentence to a new sequence of size 510 tokens + 2 required tokens
    sent = tokenizer.tokenize(ind_sent)
    original_sent.append(ind_sent)
    tokenized_sent.append(sent)
    
    ct_sent_length = len(sent)
    if ct_sent_length <= 1:
      continue
    if (ct_sequence_length + ct_sent_length) < MAX_LEN:
      # add the current sentence to current sequence
      for item in sent:
        ct_seq_list.append (item)
      # label 2 indicates begining of a sentence
      ct_token_list.append(2)
      # label 1 is inside a sentence
      for j in range(ct_sent_length - 2):
        ct_token_list.append(1)
      # label 3 is end of a sentence
      ct_token_list.append(3)
      ct_sequence_length = ct_sequence_length + ct_sent_length 

      # for debugging - can remove now
      if (len(ct_token_list) != len(ct_seq_list)):
        print ('mistmatch in length', i)
        print (ct_seq_list)
        print (ct_token_list)
        print ('labels ', len(ct_token_list))
        print ('word-tokens ', len(ct_seq_list))
  #    ct_sequence_length + ct_sent_length more than max_length. split sentence
  # add one part to current sequence, another to next sequence
    else:
      # the split sentence is more than 510
      num_left_in_ct_seq = MAX_LEN - ct_sequence_length
      first_part_ct_sent = sent[ :num_left_in_ct_seq]
      last_part_ct_sent = sent [ num_left_in_ct_seq : ]

      # following very special case - a second split is needed


      # add first part to ct_seq_list
      for item2 in first_part_ct_sent:
        ct_seq_list.append (item2)
      ct_token_list.append(2)
      for k in range(num_left_in_ct_seq - 1):
        ct_token_list.append(1)


        ## deal with exceptions last part is null
      if len(last_part_ct_sent) == 0:
        ct_token_list[-1] = 3
        #print ('zero len', len(ct_seq_list))
        mas_sequence_list.append(ct_seq_list)
        mas_label_list.append(ct_token_list)
        ct_seq_list = []
        ct_token_list = []
        ct_sequence_length = 0
      elif len(last_part_ct_sent) == 1:
        #print ('one len', len(ct_seq_list))
        mas_sequence_list.append(ct_seq_list)
        mas_label_list.append(ct_token_list)
        ct_seq_list = []
        ct_token_list = []
        for item4 in last_part_ct_sent:
          ct_seq_list.append (item4)
        ct_token_list.append(3)
        ct_sequence_length = 1
      else:
        #print ('normal len', len(ct_seq_list))
        mas_sequence_list.append(ct_seq_list)
        mas_label_list.append(ct_token_list)
        ct_seq_list = []
        ct_token_list = []
        if len(last_part_ct_sent) < MAX_LEN:
          for item3 in last_part_ct_sent:
            ct_seq_list.append (item3)
            ct_token_list.append(1)
          ct_token_list[-1] = 3
          ct_sequence_length = len(last_part_ct_sent)
        else:     # need to split again.
          ## there are 3 sentences that require multiple splits
          ## iterate   len (last_part_ct_sent) / MAX_LEN
          num_iter = len(last_part_ct_sent) // MAX_LEN
          last_segment = len(last_part_ct_sent) % MAX_LEN
          #print('split sentence iter  last_seg ', num_iter, last_segment)
          # iterate num_iter times to split large into 510 chunks
          for si in range (num_iter):
            offset = si * MAX_LEN
            for si_i in range (MAX_LEN):
              ct_seq_list.append(last_part_ct_sent[offset+si_i])
            ct_token_list.append(2)
            for si_j in range(MAX_LEN -2 ):
              ct_token_list.append(1)
            ct_token_list.append(3)
            mas_sequence_list.append(ct_seq_list)
            mas_label_list.append(ct_token_list)
            ct_seq_list = []
            ct_token_list = []
          # add the last segment
          ct_seq_list = last_part_ct_sent[-last_segment: ]
          for si_k in range (last_segment -1 ):
            ct_token_list.append(1)
          ct_token_list.append(3)
          mas_sequence_list.append(ct_seq_list)
          mas_label_list.append(ct_token_list)
          ct_sequence_length = last_segment

  mas_sequence_list.append(ct_seq_list)
  mas_label_list.append(ct_token_list)
  return mas_sequence_list, mas_label_list, original_sent, tokenized_sent


In [15]:
def create_attention_masks(master_seq_list):
  # Create attention masks
  attention_masks = []

  # For each sentence...
  for sent in master_seq_list :
      
      # Create the attention mask.
      #   - If a token ID is 0, then it's padding, set the mask to 0.
      #   - If a token ID is > 0, then it's a real token, set the mask to 1.
      att_mask = [int(token_id > 0) for token_id in sent]
      
      # Store the attention mask for this sentence.
      attention_masks.append(att_mask)

  return attention_masks


In [16]:
def preprocess_small():

  extracted_sentences = create_sentences()
  mas_sequence_list, mas_label_list, original_sent, tokenized_sent = concatenate_sentences(extracted_sentences)
  master_label_list = []

  for line in mas_label_list:
    #add a padding token to beginning and end
    tmp_list = []
    tmp_list.append(0)
    for tok in line:
      tmp_list.append(tok)
    tmp_list.append(0)
    master_label_list.append(tmp_list)

  master_seq_list = []
  for line_str in mas_sequence_list:
    #add the BERT token for [CLP] and [SEP] to the sentences
    tmp_list = []
    tmp_list.append(101)
    for str_tok in line_str:
      tmp_list.append(tokenizer.convert_tokens_to_ids(str_tok))
    tmp_list.append(102)
    master_seq_list.append(tmp_list)

  MAX_LEN = 510

  for ri in range (len(master_seq_list[-1]), MAX_LEN+2):
    master_seq_list[-1].append(0)

  for rj in range (len(master_label_list[-1]), MAX_LEN+2):
    master_label_list[-1].append(0)

  attention_masks = create_attention_masks(master_seq_list)

  return master_seq_list, master_label_list, attention_masks, original_sent, tokenized_sent



In [17]:
def create_train_val(seq_list, labels, masks):
  # Use train_test_split to split our data into train and validation sets for
  # training
  # from sklearn.model_selection import train_test_split

  # Use 85% for training and 15% for validation.
  train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(seq_list, labels, 
                                                              random_state=seed, test_size=0.15)
  # Do the same for the masks.
  train_masks, validation_masks, _, _ = train_test_split(masks, labels,
                                              random_state=seed, test_size=0.15)# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

  return train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks


In [18]:
def create_tensors(train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks):
  train_inputs = torch.tensor(train_inputs)
  validation_inputs = torch.tensor(validation_inputs)

  train_labels = torch.tensor(train_labels)
  validation_labels = torch.tensor(validation_labels)

  train_masks = torch.tensor(train_masks)
  validation_masks = torch.tensor(validation_masks)

  return train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks



In [19]:
def run_tensors():
  master_seq_list_small, master_label_list_small, attention_masks_small, original_sents, tokenized_sents = preprocess_small()
  train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = create_train_val(master_seq_list_small,master_label_list_small,attention_masks_small)
  train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = create_tensors(train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks)

  return train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks

In [20]:
def create_train_loader(train_inputs, train_labels, train_masks, batch_size):
  # Create the DataLoader for our training set.
  train_data = TensorDataset(train_inputs, train_masks, train_labels)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, batch_size=batch_size,sampler=train_sampler)

  return train_dataloader

In [21]:
def create_val_loader(validation_inputs, validation_labels, validation_masks, batch_size):
  # Create the DataLoader for our validation set.
  validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
  validation_sampler = SequentialSampler(validation_data)
  validation_dataloader = DataLoader(validation_data, batch_size=batch_size, sampler=validation_sampler)

  return validation_dataloader

In [22]:
# from seqeval.metrics import f1_score
master_mismatch_listOfLists = []

def flat_accuracy(preds, labels, att_mask, input_tokens):
    total_count = 0
    match_count = 0
    mismatch = False
    temp_list = []
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    att_flat = att_mask.flatten ()
    input_tkns_flat = input_tokens.flatten()

    for i, j_flag in enumerate(att_flat):
      if j_flag > 0:
        total_count = total_count + 1
        if pred_flat[i] == labels_flat[i]:
          match_count = match_count + 1
        else:
          mismatch = True
    if mismatch == True:
      input_id_list = input_tkns_flat.tolist() 
      temp_list.append(tokenizer.convert_ids_to_tokens(input_id_list))
      temp_list.append(pred_flat.tolist())
      temp_list.append (labels_flat.tolist())
      master_mismatch_listOfLists.append(temp_list)
    return match_count / total_count

#flat_accuracy(logits, label_ids, attention_ids, input_token_ids)

# Functions to train the model

In [23]:

max_grad_norm = 1.0
device = torch.device("cuda")


def pretrain(model,epochs, batch, learning_rate1, decay, weights=None):
  model = model
  model.cuda()
  model.train()

  train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = run_tensors()
  train_dataloader = create_train_loader(train_inputs,train_labels,train_masks,batch)
  optimizer = AdamW(model.parameters(),betas=(0.9,0.999), lr=learning_rate1,weight_decay=decay, eps=1e-6) 

  
  for name, param in model.named_parameters():
    if 'classifier' not in name: # classifier layer
      param.requires_grad = False

  for _ in trange(epochs, desc="Epoch"):
      # TRAIN loop
      model.train()
      tr_loss = 0
      nb_tr_examples, nb_tr_steps = 0, 0
      for step, batch in enumerate(train_dataloader):
          # add batch to gpu
          batch = tuple(t.to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch
          # forward pass
          outputs = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask, labels=b_labels)
          
          # model outputs are always tuples in pytorch-transformers
          # loss outputs[0], scores
          # model outputs are always tuples in pytorch-transformers
          # loss, cores = outputs[:2]
          loss = outputs[0]
          # backward pass
          loss.backward()
          # track train loss
          tr_loss += loss.item()
          nb_tr_examples += b_input_ids.size(0)
          nb_tr_steps += 1
          # gradient clipping
          torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
          # update parameters
          optimizer.step()
          model.zero_grad()
      # print train loss per epoch
      #print("Train loss: {}".format(tr_loss/nb_tr_steps))

  return model

  

In [24]:
import numpy as np
#epochs = 5
max_grad_norm = 1.0
device = torch.device("cuda")


def train(model,epochs,  batch, learning_rate1, learning_rate2,decay):

  model = pretrain(model,1, batch, learning_rate1, decay)

  # Set bert parameters back to trainable
  for name, param in model.named_parameters():
      param.requires_grad = True


  train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = run_tensors()
  train_dataloader = create_train_loader(train_inputs,train_labels,train_masks,batch)
  optimizer = AdamW(model.parameters(),betas=(0.9,0.999), lr=learning_rate2,weight_decay=decay, eps=1e-6) 

  for _ in trange(epochs, desc="Epoch"):
      # TRAIN loop
      model.train()
      tr_loss = 0
      nb_tr_examples, nb_tr_steps = 0, 0
      for step, batch in enumerate(train_dataloader):
          # add batch to gpu
          batch = tuple(t.to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch
          # forward pass
          outputs = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask, labels=b_labels)
          
          # model outputs are always tuples in pytorch-transformers
          # loss outputs[0], scores
          # model outputs are always tuples in pytorch-transformers
          # loss, cores = outputs[:2]
          loss = outputs[0]
          # backward pass
          loss.backward()
          # track train loss
          tr_loss += loss.item()
          nb_tr_examples += b_input_ids.size(0)
          nb_tr_steps += 1
          # gradient clipping
          torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
          # update parameters
          optimizer.step()
          model.zero_grad()
      # print train loss per epoch
      #print("Train loss: {}".format(tr_loss/nb_tr_steps))

  return model

#train(model)    
  

In [25]:
def validate(model, batch):
  # VALIDATION on validation set
  model.eval()
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  predictions , true_labels = [], []
  attention_ids, input_token_ids = [], []
  train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = run_tensors()
  validation_dataloader = create_train_loader(validation_inputs,validation_labels,validation_masks,batch)
  for batch in validation_dataloader:
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_labels = batch
      
      with torch.no_grad():
          output1 = model(b_input_ids, token_type_ids=None,
                                attention_mask=b_input_mask, labels=b_labels)
          output2 = model(b_input_ids, token_type_ids=None,
                          attention_mask=b_input_mask)
      tmp_eval_loss = output1[0]
      logits = output2[0]
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
      attention_ids = b_input_mask.to('cpu').numpy()
      input_token_ids = b_input_ids.to('cpu').numpy()
      predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
      true_labels.append(label_ids)
      
      tmp_eval_accuracy = flat_accuracy(logits, label_ids, attention_ids, input_token_ids)
      # identify mismatched texts

      eval_loss += tmp_eval_loss.mean().item()
      eval_accuracy += tmp_eval_accuracy
      
      nb_eval_examples += b_input_ids.size(0)
      nb_eval_steps += 1

  eval_loss = eval_loss/nb_eval_steps
  #print("Validation loss: {}".format(eval_loss))
  #print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

  return true_labels, predictions

#true_labels, predictions = validate(model, validation_dataloader)

In [26]:
def train_model(model,epochs, batch, learning_rate1, learning_rate2, decay):
  model_train = train(model,epochs, batch, learning_rate1, learning_rate2, decay)
  
  return model_train

In [27]:
def validate_run(model, batch):
  true_labels, predictions = validate(model, batch)

  true_x = []
  for i_bat in true_labels:
    for i_sent in i_bat:
      true_x.append(i_sent)

  yy_true_list = toTags(true_x)
  xx_pred_list = toTags(predictions)  

  return f1_score(yy_true_list, xx_pred_list)

In [28]:
def toTags (listOfList) :
  retList = []
  for sent in listOfList:
    tmp_ct_list = []
    for token1 in sent:
      if (token1 == 0):
        tmp_ct_list.append('O')
      if (token1 == 2):
        tmp_ct_list.append('B-Beg')
      if (token1 == 1):
        tmp_ct_list.append('I-Inside')
      if (token1 == 3):
        tmp_ct_list.append('E-End')
      if (token1 == 4):
        tmp_ct_list.append('B-PAD')
    retList.append(tmp_ct_list)
  return retList

# Functions to fine-tune the model. 
Wandb is used to log the results. May need to create an account on wandb.ai 
If you want to use different parameters change the below dictionary with the desired values


In [ ]:
wandb.login()

sweep_config = {
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'f1',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': {
            'values': [1,2,3,4,5]
        },
        'learning_rate': {
            'values': [0.000001,0.000005,0.00001,0.00005,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1]
        },
        'batch_size': {
            'values':[4,6,8]
        },
        'learning_rate2':
        {'values':[0.000001,0.000005,0.00001,0.00005,0.0001,0.0005,0.001,0.005,0.01,0.05]},
        'weight_decay': {'values':[0.1,0.05,0.01]}
    }
}

If you want to save the fine-tuning process in your wandb account, add the parameter entity="your account name" to the sweep_id

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="SBD pretraining")

In [ ]:
best_f1 = 0

def finetune(config=None):
  global best_f1
  default_config = {"learning_rate1":1e-3,
                    "learning_rate2":1e-3,
                  "epochs":2,
                  "batch_size":6,
                  "weight_decay":0.01}
  wandb.init(config=default_config)
  config = wandb.config

  model_val = train_model(model,config.epochs, config.batch_size, config.learning_rate1, config.learning_rate2,config.weight_decay)      
  f1 = validate_run(model_val, config.batch_size)
  if f1> best_f1:
    torch.save(model_val, "best_model")
    print("model saved")
    best_f1 = f1
  wandb.log({"f1":f1})


sweep_id = wandb.sweep(sweep_config, project="SBD pretraining")
wandb.agent(sweep_id,finetune)
        
